# Part 1

In [11]:
import pandas as pd
import datetime as datetime
import numpy as np

In [12]:
filename = 'input.txt'
with open(filename) as f:
    data = f.readlines()
f.close()
for n,line in enumerate(data):
    data[n] = line.rstrip()

In [13]:
data

['[1518-07-28 00:10] falls asleep',
 '[1518-10-16 00:04] Guard #479 begins shift',
 '[1518-07-18 00:55] falls asleep',
 '[1518-05-03 00:57] wakes up',
 '[1518-10-11 00:04] Guard #3391 begins shift',
 '[1518-03-15 00:40] falls asleep',
 '[1518-10-09 00:56] wakes up',
 '[1518-08-02 00:25] falls asleep',
 '[1518-05-16 00:41] wakes up',
 '[1518-07-03 00:47] falls asleep',
 '[1518-08-08 00:03] falls asleep',
 '[1518-06-17 00:04] Guard #1951 begins shift',
 '[1518-03-12 00:40] falls asleep',
 '[1518-03-13 23:57] Guard #1579 begins shift',
 '[1518-06-29 00:08] falls asleep',
 '[1518-09-05 23:59] Guard #79 begins shift',
 '[1518-04-15 00:27] wakes up',
 '[1518-03-21 00:03] Guard #1951 begins shift',
 '[1518-09-29 00:45] wakes up',
 '[1518-06-22 00:00] Guard #479 begins shift',
 '[1518-09-22 00:12] falls asleep',
 '[1518-11-11 00:39] wakes up',
 '[1518-06-18 00:01] Guard #79 begins shift',
 '[1518-11-10 00:36] wakes up',
 '[1518-05-24 23:56] Guard #191 begins shift',
 '[1518-08-08 00:07] wakes 

In [14]:
time = data[0].replace('[','').split(']')[0]
datetime_object = datetime.strptime(time, '%Y-%m-%d %H:%M')

dates = [datetime.strptime(row.replace('[','').split(']')[0], '%Y-%m-%d %H:%M') for row in data]
states = [row.replace('[','').split(']')[1].replace('#',' ') for row in data]

In [15]:
df = pd.DataFrame()
df['dates'] = dates
df['states'] = states

df.sort_values('dates', inplace=True)
df.head()

,dates,states
394,1518-03-10 23:57:00,Guard 73 begins shift
322,1518-03-11 00:06:00,falls asleep
32,1518-03-11 00:22:00,wakes up
479,1518-03-11 00:38:00,falls asleep
71,1518-03-11 00:41:00,wakes up


In [16]:
ids = []
new_states = [] #Wake = 0, Asleep = 1
for each in df['states'].values:
    current_id = [int(s) for s in each.split() if s.isdigit()]
    if len(current_id) > 0:
        ids.append(current_id[0])
    else:
        ids.append(ids[-1])
    
    if each[1] == 'G' or each[1] == 'w':
        new_states.append(0)
    else:
        new_states.append(1)

df['states'] = new_states
df['guard_id'] = ids
df.reset_index(inplace=True)
df.drop(columns=['index'],inplace=True)
df.head(10)



,dates,states,guard_id
0,1518-03-10 23:57:00,0,73
1,1518-03-11 00:06:00,1,73
2,1518-03-11 00:22:00,0,73
3,1518-03-11 00:38:00,1,73
4,1518-03-11 00:41:00,0,73
5,1518-03-12 00:01:00,0,2833
6,1518-03-12 00:15:00,1,2833
7,1518-03-12 00:30:00,0,2833
8,1518-03-12 00:40:00,1,2833
9,1518-03-12 00:44:00,0,2833


In [17]:
def analyze_guard(current_id, df):
    tmp_df = df.query('guard_id == @current_id').reset_index()

    schedule = np.zeros(60)

    for index, row in tmp_df.iterrows():
        if row['states'] == 1:
            first_minute = tmp_df.iloc[index]['dates'].minute
            last_minute = tmp_df.iloc[index+1]['dates'].minute
            minutes_asleep = (tmp_df.iloc[index+1]['dates'] - tmp_df.iloc[index]['dates'])/60
        
            if first_minute < last_minute:
                schedule[list(range(first_minute,last_minute))] += 1
        
            else:
                first_minute = 0
                schedule[list(range(first_minute,last_minute))] += 1
    
    return schedule

In [18]:
analyze_guard(73, df)

array([ 0.,  0.,  0.,  1.,  1.,  1.,  2.,  3.,  4.,  4.,  4.,  4.,  4.,
        4.,  4.,  5.,  7.,  8.,  9.,  9., 10.,  9.,  8.,  9.,  9., 10.,
       10., 11., 11., 11., 11., 12., 11., 10., 10., 10., 10., 12., 13.,
       13., 13., 11., 11., 11., 14., 13., 12., 11., 11., 10., 11., 11.,
       11., 10.,  7.,  5.,  3.,  2.,  0.,  0.])

In [19]:
list_of_ids = df['guard_id'].unique()

guard_schedules = []
guard_sleeptime = []

for each_guard_id in list_of_ids:
    schedule = analyze_guard(each_guard_id,df)
    sleeptime = np.sum(schedule)
    
    guard_schedules.append(schedule)
    guard_sleeptime.append(sleeptime)

In [20]:
guard_characteristics = []

for i,each in enumerate(list_of_ids):
    guard_characteristics.append((each, guard_sleeptime[i], guard_schedules[i]))

In [21]:
result = guard_characteristics[np.argmax(guard_sleeptime)][0]*np.argmax(guard_characteristics[np.argmax(guard_sleeptime)][2])
print(result)

3212


In [22]:
guard_characteristics

[(73,
  461.0,
  array([ 0.,  0.,  0.,  1.,  1.,  1.,  2.,  3.,  4.,  4.,  4.,  4.,  4.,
          4.,  4.,  5.,  7.,  8.,  9.,  9., 10.,  9.,  8.,  9.,  9., 10.,
         10., 11., 11., 11., 11., 12., 11., 10., 10., 10., 10., 12., 13.,
         13., 13., 11., 11., 11., 14., 13., 12., 11., 11., 10., 11., 11.,
         11., 10.,  7.,  5.,  3.,  2.,  0.,  0.])),
 (2833,
  201.0,
  array([1., 1., 1., 1., 1., 1., 1., 3., 2., 2., 2., 2., 2., 2., 3., 4., 4.,
         4., 4., 5., 5., 6., 6., 6., 5., 5., 5., 5., 5., 5., 4., 4., 4., 4.,
         4., 4., 4., 4., 4., 4., 5., 4., 4., 4., 4., 4., 4., 4., 4., 3., 3.,
         3., 3., 3., 3., 3., 3., 1., 0., 0.])),
 (709,
  415.0,
  array([ 1.,  1.,  1.,  2.,  2.,  4.,  5.,  4.,  4.,  4.,  4.,  7.,  7.,
          7.,  9.,  9., 10., 11., 11., 11., 11., 12., 12., 11., 11., 12.,
         12., 11., 11., 11., 10., 10., 10., 10.,  9., 10.,  8.,  8.,  8.,
          7.,  7.,  6.,  6.,  7.,  7.,  7.,  7.,  5.,  5.,  5.,  5.,  6.,
          6.,  5.,  4.,  4., 

# Part 2

In [23]:
sum_of_minutes = []

for i,each_guard in enumerate(guard_characteristics):
    sum_of_minutes.append(each_guard[2])

sum_of_minutes = np.array(sum_of_minutes)

In [24]:

highest_ratio = 0
for each_minute in np.argmax(sum_of_minutes,axis=1):
    most_time_asleep = np.max(sum_of_minutes[:,each_minute])
    second_most_time_asleep = np.partition(sum_of_minutes[:,each_minute].flatten(), -2)[-2]
    if second_most_time_asleep != 0:
        ratio = most_time_asleep/second_most_time_asleep
    else:
        ratio = 0
    
    if ratio > highest_ratio:
        highest_ratio = ratio
        highest_minute = each_minute
        highest_guard = np.argmax(sum_of_minutes[:,each_minute])

In [25]:
print(highest_ratio, highest_minute, guard_characteristics[highest_guard][0])

1.4166666666666667 26 191
